<a href="https://colab.research.google.com/github/Kurt9806/Seismic-analysis/blob/main/Analisis_pseudo3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Pseudo Tridimensional

Para utilizar este codigo, se requiere ya realizar el calculo de las rigideces, el calculo de las masas, de los momentos polares y subir una tabla con todos los datos.

In [1]:
import pandas as pd
import numpy as np
import math

In [7]:
df = pd.read_csv('Semana 6.csv')
print(df)
print(df.iloc[1,-2])

  Descripcion            Kl           K11  ...           cos  sen         r
0     Port. 1   15095.18411   30190.36822  ...  1.000000e+00    0  5.776107
1     Port. 2   15095.18411   30190.36822  ...  1.000000e+00    0 -0.723893
2     Port. 3  173091.44450  346182.88890  ...  1.000000e+00    0 -7.223893
3     Port. A   15095.18411   30190.36822  ...  6.125740e-17    1 -5.816010
4     Port. B   15095.18411   30190.36822  ...  6.125740e-17    1  0.683990
5     Port. C  173091.44450  346182.88890  ...  6.125740e-17    1  7.183990

[6 rows x 12 columns]
0


In [9]:
# Codigo de ensamblaje de la matriz de rigidez
K11 = np.zeros((3,3)); K12=np.zeros((3,3)); K22=np.zeros((3,3));

for i in range(len(df)):
  k11 = df.iloc[i,2]
  k12 = df.iloc[i,3]
  k22 = df.iloc[i,4]
  sen = df.iloc[i,-2]
  cos = df.iloc[i,-3]
  r = df.iloc[i,-1]

  A = np.array([[cos**2, cos*sen, r*cos],[cos*sen, sen**2, r*sen],[r*cos,r*sen,r**2]])

  K11 = K11 + A*k11
  K12 = K12 + A*k12
  K22 = K22 + A*k22

KL1 = np.append(K11,K12,axis=1)
KL2 = np.append(K12,K22,axis=1)
K = np.append(KL1,KL2,axis=0)

print(K)
print(K.shape)

[[ 4.06563625e+05  2.49050306e-11 -2.34826011e+06 -2.03281813e+05
  -1.24525153e-11  1.17413006e+06]
 [ 2.49050306e-11  4.06563625e+05  2.33203673e+06 -1.24525153e-11
  -2.03281813e+05 -1.16601837e+06]
 [-2.34826011e+06  2.33203673e+06  3.79902429e+07  1.17413006e+06
  -1.16601837e+06 -1.89951215e+07]
 [-2.03281813e+05 -1.24525153e-11  1.17413006e+06  2.03281813e+05
   1.24525153e-11 -1.17413006e+06]
 [-1.24525153e-11 -2.03281813e+05 -1.16601837e+06  1.24525153e-11
   2.03281813e+05  1.16601837e+06]
 [ 1.17413006e+06 -1.16601837e+06 -1.89951215e+07 -1.17413006e+06
   1.16601837e+06  1.89951215e+07]]
(6, 6)


In [12]:
# Calculo de la matriz de masa
m1 = 14.8010703363914
m2=13.1741590214067; J=356.884739098682;

M1 = np.array([[m1,0,0],[0,m1,0],[0,0,J]])
M2 = np.array([[m2,0,0],[0,m2,0],[0,0,J]])
M11 = np.append(M1,np.zeros((3,3)),axis=1)
M12 = np.append(np.zeros((3,3)),M2,axis=1)
M = np.append(M11,M12,axis=0)
print(M)
print(M.shape)

[[ 14.80107034   0.           0.           0.           0.
    0.        ]
 [  0.          14.80107034   0.           0.           0.
    0.        ]
 [  0.           0.         356.8847391    0.           0.
    0.        ]
 [  0.           0.           0.          13.17415902   0.
    0.        ]
 [  0.           0.           0.           0.          13.17415902
    0.        ]
 [  0.           0.           0.           0.           0.
  356.8847391 ]]
(6, 6)


# Analisis Modal

In [13]:
from scipy.linalg import eig

l_ambda, X = eig(K,M)
omega = np.sqrt(l_ambda)
omega = np.real(omega)
print(omega)
print(X)

[409.31087272 192.87907599 157.09414687  36.96097797  94.83258809
  75.47539523]
[[ 5.34671770e-01 -5.74906785e-01 -3.09160324e-01  3.75951245e-01
  -5.83444702e-01  3.75956650e-01]
 [-5.30977894e-01 -5.78906263e-01  3.07024434e-01 -3.73353918e-01
   5.79413869e-01  3.78572084e-01]
 [-3.35664041e-01  3.24140159e-14  2.03277787e-01  4.94471063e-02
  -7.50701955e-02  4.18423708e-14]
 [-3.72892409e-01  4.07450508e-01 -5.80077519e-01  5.98142023e-01
   3.98389039e-01  5.95978460e-01]
 [ 3.70316215e-01  4.10285036e-01  5.76069948e-01 -5.94009651e-01
  -3.95636697e-01  6.00124528e-01]
 [ 2.08171512e-01  5.13586500e-14  3.16997665e-01  7.87473886e-02
   5.47182915e-02 -2.91069299e-14]]


In [18]:
# Otra forma de mostrar los modos
T = 2*math.pi/omega
# Sort the vector matrix and modes to minimum and maximum
indx = T.argsort(axis=0)
T = T[indx]
X = X[ : , indx ]


Modos = pd.DataFrame(data=X,columns=T)

print(Modos)
print(Modos.iloc[:,0])

   0.015351      0.032576  0.039996      0.066256  0.083248  0.169995
0  0.534672 -5.749068e-01 -0.309160  3.759567e-01  0.375951 -0.583445
1 -0.530978 -5.789063e-01  0.307024  3.785721e-01 -0.373354  0.579414
2 -0.335664  3.241402e-14  0.203278  4.184237e-14  0.049447 -0.075070
3 -0.372892  4.074505e-01 -0.580078  5.959785e-01  0.598142  0.398389
4  0.370316  4.102850e-01  0.576070  6.001245e-01 -0.594010 -0.395637
5  0.208172  5.135865e-14  0.316998 -2.910693e-14  0.078747  0.054718
0    0.534672
1   -0.530978
2   -0.335664
3   -0.372892
4    0.370316
5    0.208172
Name: 0.01535064354739582, dtype: float64


In [20]:
# Calcular el factor de participacion
FPM = np.zeros((6,6))

for i in range(len(Modos)):
  phi = Modos.iloc[:,i].to_numpy()
  Numerator = np.matmul(np.transpose(phi),M)
  Denominator = np.matmul(np.matmul(np.transpose(phi),M),phi)
  FPM [:,i] = Numerator/Denominator

print(FPM)
print(FPM.shape)

[[ 1.16861485e-01 -5.96841434e-01 -7.35387330e-02  4.08040494e-01
   3.35155878e-01 -5.00897868e-01]
 [-1.16054126e-01 -6.00993506e-01  7.30306774e-02  4.10879127e-01
  -3.32840393e-01  4.97437325e-01]
 [-1.76898365e+00  8.11389207e-13  1.16588890e+00  1.09500633e-12
   1.06289668e+00 -1.55400311e+00]
 [-7.25433267e-02  3.76501034e-01 -1.22814092e-01  5.75739248e-01
   4.74623687e-01  3.04429436e-01]
 [ 7.20421480e-02  3.79120255e-01  1.21965609e-01  5.79744517e-01
  -4.71344663e-01 -3.02326230e-01]
 [ 1.09708505e+00  1.28561220e-12  1.81812320e+00 -7.61722436e-13
   1.69272469e+00  1.13270513e+00]]
(6, 6)


In [24]:
# Calculando la pseudo aceleracion
Sa = [0.546971167,0.610338114,0.637636389,0.734237605,0.796749058,1.115869474]
Sd = Sa/omega**2
print(Sd)

[3.26480969e-06 1.64059075e-05 2.58376544e-05 5.37464416e-04
 8.85944166e-05 1.95885642e-04]


In [32]:
# Vector de desplazamiento 
# En X
X_x = Modos.iloc[0:4,:].to_numpy()
X_x = np.delete(X_x,(1,2),axis=0)
X_x = np.transpose(X_x)
FPM_x = np.delete(FPM[0:4,:],(1,2),axis=0)
FPM_x = np.transpose(FPM_x)

# Calcular los desplazamientos por piso y por modo
u = np.array([FPM_x[j]*Sd[j]*X_x[j] for j in range(len(X_x))])

# Combinarlo 
u_abs = 0.25*u.sum(axis=0) + 0.75*np.sum(u**2, axis=0)
print(u_abs)

# Desplazamientos relativos
u_rel = np.append([u_abs[0]],np.diff(u_abs))
print(u_rel)

# Verificacion segun la norma
delta_max = 0.007*3.5
Verif = [u_rel>=delta_max]
print(Verif)

[3.93278489e-05 5.94697973e-05]
[3.93278489e-05 2.01419484e-05]
[array([False, False])]


In [35]:
# Cortante por piso
# En X 
Mp = [m1,m2]
print(Mp)

V = np.array([FPM_x[j]*Sa[j]*X_x[j]*Mp for j in range(len(X_x))])
#Combinacion
V_abs = 0.25*u.sum(axis=0) + 0.75*np.sum(u**2, axis=0)
print(V_abs)

[14.8010703363914, 13.1741590214067]
[3.93278489e-05 5.94697973e-05]
